In [1]:
!pip install sqlalchemy psycopg2 pandas


[notice] A new release of pip available: 22.3 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [2]:
from sqlalchemy import create_engine, MetaData
import pandas as pd
import json
from sqlalchemy import create_engine, text

user = "padel_user"
password = "portopadelleague"
host = "35.205.246.86"  # your GCE VM public IP
port = "5432"
database = "padel_league"

# PostgreSQL destination
pg_engine = create_engine(f"postgresql://{user}:{password}@{host}:{port}/{database}")


In [3]:
def get_all_tables():
    """
    Retrieves a list of all table names in the connected PostgreSQL database.

    Returns:
        list: A list of table names (str).
    """
    with pg_engine.connect() as connection:
        result = connection.execute(text("""
            SELECT table_name 
            FROM information_schema.tables 
            WHERE table_schema = 'public' AND table_type = 'BASE TABLE'
        """))
        tables = [row[0] for row in result.fetchall()]
        return tables

In [4]:
def get_columns_and_types(table_name: str):
    """
    Retrieves all column names and their data types from a specific table.

    Args:
        table_name (str): The name of the table.

    Returns:
        list of tuples: Each tuple contains (column_name, data_type).
    """
    with pg_engine.connect() as connection:
        result = connection.execute(text("""
            SELECT column_name, data_type
            FROM information_schema.columns
            WHERE table_name = :table
        """), {"table": table_name})
        
        columns = [(row[0], row[1]) for row in result.fetchall()]
        return columns

In [5]:
def add_column_to_table(table_name: str, column_name: str, column_type: str):
    """
    Adds a new column to an existing PostgreSQL table.

    Args:
        table_name (str): The name of the table to modify.
        column_name (str): The name of the new column.
        column_type (str): The SQL type of the new column (e.g., 'TEXT', 'INTEGER', 'BOOLEAN', etc.).
    """
    with pg_engine.begin() as connection:
        # Check if column already exists
        check_query = text(f"""
            SELECT column_name 
            FROM information_schema.columns 
            WHERE table_name = :table AND column_name = :column
        """)
        result = connection.execute(check_query, {"table": table_name, "column": column_name}).fetchone()

        if result:
            print(f"⚠️ Column '{column_name}' already exists in table '{table_name}'.")
        else:
            alter_query = text(f'ALTER TABLE "{table_name}" ADD COLUMN "{column_name}" {column_type}')
            connection.execute(alter_query)
            print(f"✅ Added column '{column_name}' of type '{column_type}' to table '{table_name}'.")


In [6]:
def fill_column_sequentially(table_name: str, column_name: str):
    """
    Fills a specified column in a table with sequential integer values starting from 1.

    Args:
        table_name (str): The name of the table.
        column_name (str): The name of the column to fill.
    """
    with pg_engine.begin() as connection:
        # PostgreSQL-specific SQL using CTE to generate row numbers
        update_query = text(f"""
            WITH numbered AS (
                SELECT ctid, ROW_NUMBER() OVER () as seq
                FROM "{table_name}"
            )
            UPDATE "{table_name}" AS t
            SET "{column_name}" = n.seq
            FROM numbered n
            WHERE t.ctid = n.ctid
        """)
        connection.execute(update_query)
        print(f"✅ Filled column '{column_name}' in table '{table_name}' with sequential values.")


In [7]:
def fix_missing_sequences():
    """
    Ensures all tables with an 'id' column have an associated sequence
    and sets the default value of the 'id' column to use the sequence.
    """
    tables = get_all_tables()
    with pg_engine.begin() as connection:
        for table in tables:
            # Check if the table has an 'id' column
            id_column_check = connection.execute(text(f"""
                SELECT column_name, data_type 
                FROM information_schema.columns 
                WHERE table_name = :table AND column_name = 'id'
            """), {"table": table}).fetchone()
            
            if not id_column_check:
                continue

            col_name, data_type = id_column_check
            if data_type not in ['integer', 'bigint']:
                continue  # only process integer/bigint ids

            sequence_name = f"{table}_id_seq"

            print(f"Fixing sequence for: {table}")

            # Create the sequence if it doesn't exist
            connection.execute(text(f"""
                CREATE SEQUENCE IF NOT EXISTS {sequence_name}
            """))

            # Set the default value of the id column to use the sequence
            connection.execute(text(f"""
                ALTER TABLE {table} ALTER COLUMN id SET DEFAULT nextval('{sequence_name}')
            """))

            # Set sequence to current MAX(id), or fallback to 1
            connection.execute(text(f"""
                SELECT setval('{sequence_name}', GREATEST((SELECT COALESCE(MAX(id), 0) FROM {table}), 1))
            """))

            print(f"✅ Sequence set for {table}")
        return True

In [8]:
def get_all_elements_from_table(table_name, order_by="id", descending=True):
    """
    Retrieves all rows from the specified table in the PostgreSQL database,
    optionally ordered by a column.

    Args:
        table_name (str): The name of the table to retrieve data from.
        order_by (str): Column name to order by (default is 'id').
        descending (bool): If True, order descending (newest first).

    Returns:
        list: A list of dictionaries representing rows in the table.
    """
    order_clause = f"ORDER BY {order_by} {'DESC' if descending else 'ASC'}"
    with pg_engine.connect() as connection:
        result = connection.execute(text(f"SELECT * FROM {table_name} {order_clause}"))
        rows = result.mappings().all()
        return rows

In [9]:
def delete_elements_with_condition(table_name, column, value):
    """
    Deletes rows from a specified table where a column matches a given value.

    Args:
        table_name (str): The name of the table.
        column (str): The column name to apply the condition.
        value (Any): The value to match for deletion.

    Returns:
        int: Number of rows deleted.
    """
    with pg_engine.begin() as connection:
        result = connection.execute(
            text(f"DELETE FROM {table_name} WHERE {column} = :val"),
            {"val": value}
        )
        return result.rowcount

In [10]:
get_all_tables()

['product_attribute_values',
 'product_attributes',
 'product_images',
 'registrations',
 'backend_app',
 'players_in_match',
 'product_attribute_values_in_product',
 'product_attributes_in_product',
 'players_in_division',
 'imageables',
 'images',
 'divisions',
 'editions',
 'leagues',
 'matches',
 'order_lines',
 'orders',
 'users',
 'players',
 'products',
 'news']

In [11]:
def delete_null_match_rows(table_name, player_id):
    """
    Deletes a row for a player where match_id and team are null.

    Args:
        table_name (str): The table name to delete from.
        player_id (int): The player ID to match.

    Returns:
        int: Number of rows deleted.
    """
    with pg_engine.connect() as connection:
        result = connection.execute(
            text(f"""
                DELETE FROM {table_name}
                WHERE player_id = :player_id AND match_id IS NULL AND team IS NULL
            """),
            {"player_id": player_id}
        )
        return result.rowcount
    
delete_null_match_rows("players_in_match", 26)

1

In [22]:
delete_elements_with_condition("players_in_match",'id',6073)

2

In [23]:
get_all_elements_from_table('players_in_match')

[{'player_id': 42, 'match_id': 1849, 'team': 'Away', 'id': 6057},
 {'player_id': 52, 'match_id': 1849, 'team': 'Home', 'id': 6056},
 {'player_id': 54, 'match_id': 1848, 'team': 'Away', 'id': 6055},
 {'player_id': 38, 'match_id': 1848, 'team': 'Away', 'id': 6054},
 {'player_id': 55, 'match_id': 1848, 'team': 'Home', 'id': 6053},
 {'player_id': 31, 'match_id': 1848, 'team': 'Home', 'id': 6052},
 {'player_id': 54, 'match_id': 1847, 'team': 'Away', 'id': 6051},
 {'player_id': 38, 'match_id': 1847, 'team': 'Away', 'id': 6050},
 {'player_id': 52, 'match_id': 1847, 'team': 'Home', 'id': 6049},
 {'player_id': 42, 'match_id': 1846, 'team': 'Away', 'id': 6048},
 {'player_id': 55, 'match_id': 1846, 'team': 'Home', 'id': 6047},
 {'player_id': 31, 'match_id': 1846, 'team': 'Home', 'id': 6046},
 {'player_id': 54, 'match_id': 1845, 'team': 'Away', 'id': 6045},
 {'player_id': 38, 'match_id': 1845, 'team': 'Away', 'id': 6044},
 {'player_id': 42, 'match_id': 1845, 'team': 'Home', 'id': 6043},
 {'player_

In [38]:
get_columns_and_types('divisions')

[('id', 'bigint'),
 ('name', 'text'),
 ('beginning_datetime', 'timestamp without time zone'),
 ('rating', 'bigint'),
 ('end_date', 'timestamp without time zone'),
 ('logo_image_path', 'text'),
 ('large_picture_path', 'text'),
 ('has_ended', 'boolean'),
 ('open_division', 'boolean'),
 ('edition_id', 'bigint')]

In [39]:
add_column_to_table(table_name='news', column_name='latest', column_type='boolean')

✅ Added column 'latest' of type 'boolean' to table 'news'.


In [30]:
fill_column_sequentially(table_name='product_attributes_in_product', column_name='id')

✅ Filled column 'id' in table 'product_attributes_in_product' with sequential values.


In [21]:
get_columns_and_types('players_in_match')

[('player_id', 'bigint'),
 ('match_id', 'bigint'),
 ('team', 'text'),
 ('id', 'bigint')]

In [13]:
fix_missing_sequences()

Fixing sequence for: product_attribute_values
✅ Sequence set for product_attribute_values
Fixing sequence for: product_attributes
✅ Sequence set for product_attributes
Fixing sequence for: product_images
✅ Sequence set for product_images
Fixing sequence for: backend_app
✅ Sequence set for backend_app
Fixing sequence for: players_in_match
✅ Sequence set for players_in_match
Fixing sequence for: product_attribute_values_in_product
✅ Sequence set for product_attribute_values_in_product
Fixing sequence for: product_attributes_in_product
✅ Sequence set for product_attributes_in_product
Fixing sequence for: players_in_division
✅ Sequence set for players_in_division
Fixing sequence for: images
✅ Sequence set for images
Fixing sequence for: divisions
✅ Sequence set for divisions
Fixing sequence for: editions
✅ Sequence set for editions
Fixing sequence for: leagues
✅ Sequence set for leagues
Fixing sequence for: matches
✅ Sequence set for matches
Fixing sequence for: order_lines
✅ Sequence set 

True